# Development for code review: MMRDNS

## Notes
---

#### Aug 11th 2017
* Fixing the bug below helps, but now there are nans for low inclinaitons returned, and morphology of Slm are still a bit off for $(3,2,0)$.
* Bug found in the Spheroidal function:
``` C
// Incorrect
dY = pow(a2*(1.0+u),k);
```
Should be 
``` C
// Correct
dY = a2 * pow(1.0+u,k);
```
* There is definitely a noticeable error/bug in the LAL Spheroidal harmonic function: the amplitde and phase is off by ~10 percent for the q=1 test case
    * The LAL spheroidal harmonics are not properly normalized. 
    * In some cases (e.g. $(l,m)=(3,2)$) the morphology of the spheroidal harmonic function is incorrect AND not properly normalized
* Adding overall conjugation to output of Amplitude fit -- switching to correct phase convention
``` C
/*NOTE that the MATLAB code used to perform the fitting uses a different convention when handling the real and imaginary parts of psi4 than we will use here. The conjugation below makes the output of MMRDNS consistent with nrutils, which injects no manual minus signs when handling psi4, but enforces a phase convention: m>0 has frequencies >0 (non-precessing). NOTE that this may change in the future if significantly precessing systems are found to not sufficiently obey this property. See https://github.com/llondon6/nrutils_dev/blob/master/nrutils/core/nrsc.py#L1714-L1728 for more details.*/
ans = conj( ans );
```
* I've locally modified a line in the LAL Amplitude function
``` C
  /* If m<0, then take the conjugate */
  if ( input_m < 0 ) {
    /**/
    // OLD: ans = conj( ans );
    ans = pow(-1,l) * conj( ans );
  }
```

#### Aug 10th 2017
* Adding mode-by-mode comparisons and making separate section for full signal comparison
* I suspect an incorrect usage of the fninal mass fit: lalsim.E_rad_nonspinning_UIB2016(eta) --- the usage here turned out to be OK.

#### Aug 9th 2017
* (no longer of interest) The definition of time in the LAL version is correct but inconsistent with the model's construction. 
    * Tentatively changed in my version of the LAL Code by having T0=10M within the code. 
    * A better change may be to rescale the model itself with the LAL code.
* There is something incorrect about how the multipoles are being added together: when modes beyond the fundmental are added, a strange modulation is

## Setup Environment

In [1]:
# Setup ipython environment
%load_ext autoreload
%autoreload 2
# %matplotlib auto
%matplotlib inline
# Import useful things
#
# from os import system
# system('source ~/.virtual_enviroments/ringdownFD_nikhef/bin/activate')
from nrutils.core.basics import lalphenom
from kerr import rgb,gmvpfit,modelrd,anglep,qnmfit,leaver,ndflatten,sunwrap,alert,cyan,slm
from kerr.models import mmrdns as ns
from nrutils import scsearch,gwylm,FinalSpin0815,EradRational0815
# Setup plotting backend
import matplotlib as mpl
from mpl_toolkits.mplot3d import axes3d
mpl.rcParams['lines.linewidth'] = 0.8
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.size'] = 12
mpl.rcParams['axes.labelsize'] = 20
mpl.rcParams['axes.titlesize'] = 20
from matplotlib.pyplot import *
from mpl_toolkits.mplot3d import Axes3D
#
import corner
#
from numpy import *
from numpy.linalg import pinv
#
from nrutils.core.units import *
#
from scipy.optimize import minimize
from os.path import expanduser
#
import lal
from lal import MSUN_SI, PC_SI, C_SI, G_SI, MTSUN_SI
import lalsimulation as lalsim
alert('The LASIMULATION version is located at: %s'%( bold(yellow(lalsim.__path__[0])) ))
#
import review_ringdownFD_nikhef as review

(<module>)>> The LASIMULATION version is located at: /Users/book/.virtual_enviroments/ringdownFD_nikhef/opt/lalsuite/lib/python2.7/site-packages/lalsimulation


## Select and Load NR Data

In [2]:
# Find simlations to load 
A = scsearch(keyword='D11_q1.00_a0.0_m240',verbose=True)
# A = scsearch(keyword='D9_q1.2_a0.0_m160',verbose=True)
# A = scsearch(nonspinning=True,q=1.2,verbose=True,institute='gt',keyword='hrq')

(scsearch)>> Found keyword (='D11_q1.00_a0.0_m240') keyword.
(scsearch)>> Found verbose (=True) keyword.
(scsearch)>> List of keywords or string keyword found: ALL scentry objects matching will be passed. To pass ANY entries matching the keywords, input the keywords using an iterable of not of type list.
## Found 1 possibly degenerate simulations:
[0001][bradwr] HR-series: ns-q1.00	(D11_q1.00_a0.0_m240)



In [3]:
# Load simulation data
a = A[0]
y = gwylm( a, lm=[(2,2),(3,2),(4,4)], verbose=True )

(gwylm)>> Found dt (=0.15) keyword.
(gwylm)>> Found lm (=[(2, 2), (3, 2), (4, 4)]) keyword.
(gwylm)>> Found scentry_obj (=<nrutils.core.nrsc.scentry instance at 0x10b57c7e8>) keyword.
(gwylm)>> Found verbose (=True) keyword.
(validate!)>> Multiple catalog directories found. We will scan through the related list, and then store first the catalog_dir that the OS can find.
(validate!)>> Selecting "/Volumes/athena/bradwr/"
(gwylm)>> The (extraction_parameter,level) is (75.0,None), which differs from the config values of (75,0). You have either manually input the non-config values, or the handler has set them by looking at the contents of the simulation directory. 
(__make_lmlist__)>> The following spherical multipoles will be loaded:[(2, 2), (3, 2), (4, 4)]
(load)>> Loading: Ylm_WEYLSCAL4::Psi4r_l2_m2_r75.00.asc
(load)>> Imposing a default padding of 3 to the data.
(pad_wfarr)>> The shape was (11994, 3). Now the shape is (11997, 3).
(load)>> Loading: Ylm_WEYLSCAL4::Psi4r_l3_m2_r75.00.asc
(

In [4]:
#
yrd = y.ringdown(T0=10,use_peak_strain=False,verbose=True)

(ringdown)>> Time will be listed relative to the peak of luminosity.


In [5]:
# Define physical parameters for LAL comparison
M_Sol = 80
Mf_Sol = 1.00002821*M_Sol * y.mf
D_Mpc = 150
inclination = 0
fmin = 20
phi_ref = 0
#
M = M_Sol*MSUN_SI
Mf = Mf_Sol*MSUN_SI
D = D_Mpc*1.0e6*PC_SI
deltaT = physt( y.lm[2,2]['strain'].dt, M_Sol )
xf = y.xf
eta = y.eta
#
t_code = yrd.ylm[0].t

### Spot check final mass fit 

In [6]:
print '>> From LAL UIB Fit: Mf = ',1-lalsim.E_rad_nonspinning_UIB2016(eta)
print '>> From NR metadata: Mf = ',y.mf
print '>> Estimated initial mass: M_Sol = ', lalsim.Mf_to_M_nonspinning_UIB2016(eta, Mf_Sol)
print '>> Desired initial mass  : M_Sol = ', M_Sol

>> From LAL UIB Fit: Mf =  0.951588979146
>> From NR metadata: Mf =  0.951562197018
>> Estimated initial mass: M_Sol =  80.0000051654
>> Desired initial mass  : M_Sol =  80


### Directly compare amplitude functions

In [7]:
from kerr.formula.mmrdns_amplitudes import A_strain as Afitraw
for l,m,n in ns.lmn:
    Alal = lalsim.MMRDNSAmplitudeOverOmegaSquared(eta, l, m, n)
    Apy  = ns.Afit(l, m, n, eta)
    print '>> From LAL MMRDNS Fit: A%i%i%i = '%(l,m,n),Alal
    print '>> From PY  MMRDNS Fit: A%i%i%i = '%(l,m,n),Apy
    print '>> From RAWPY      Fit: A%i%i%i = '%(l,m,n), Afitraw[l,m,n](eta).conj()
    print '## Error (%) = ',100*abs((Alal-Apy)/abs(Apy))
    print '---'

>> From LAL MMRDNS Fit: A220 =  (-0.229804848082-0.0716724887031j)
>> From PY  MMRDNS Fit: A220 =  (-0.229804848082-0.0716724887031j)
>> From RAWPY      Fit: A220 =  (-0.229804848082-0.0716724887031j)
## Error (%) =  0.0
---
>> From LAL MMRDNS Fit: A221 =  (0.0386954515206-0.0465239548319j)
>> From PY  MMRDNS Fit: A221 =  (0.0386954515206-0.0465239548319j)
>> From RAWPY      Fit: A221 =  (0.0386954515206-0.0465239548319j)
## Error (%) =  0.0
---
>> From LAL MMRDNS Fit: A210 =  0j
>> From PY  MMRDNS Fit: A210 =  -0j
>> From RAWPY      Fit: A210 =  -0j
## Error (%) =  nan
---
>> From LAL MMRDNS Fit: A330 =  (-0+0j)
>> From PY  MMRDNS Fit: A330 =  0j
>> From RAWPY      Fit: A330 =  0j
## Error (%) =  nan
---
>> From LAL MMRDNS Fit: A320 =  (-0.00744566845341-0.00161945693189j)
>> From PY  MMRDNS Fit: A320 =  (-0.00744566845341-0.00161945693189j)
>> From RAWPY      Fit: A320 =  (-0.00744566845341-0.00161945693189j)
## Error (%) =  0.0
---
>> From LAL MMRDNS Fit: A440 =  (0.00854016048963-0

/Library/Python/2.7/site-packages/ipykernel/__main__.py:8: RuntimeWarning: invalid value encountered in cdouble_scalars


## Compare Spheroidal Harmonic Normalization

In [10]:
#
from kerr import ssprod
#
for l,m,n in ns.lmn:

    #
    kappa = lalsim.KAPPA(xf,l,m)
    Nlal = lalsim.spheroidalHarmonicNormalization(kappa,l,m,n)

    #
    z = (l,m,n)
    Npy = sqrt( ssprod(xf,z,z) )

    #
    print '---'
    print '>> From LAL: N%i%i%i = '%(l,m,n),Nlal
    print '>> From  PY: N%i%i%i = '%(l,m,n),Npy
    print '## Error (%) = ',100*abs((Nlal-Npy)/abs(Npy))

---
>> From LAL: N220 =  6.6911139222
>> From  PY: N220 =  (6.69148591916+0j)
## Error (%) =  0.00555925794624
---
>> From LAL: N221 =  6.66837225332
>> From  PY: N221 =  (6.66928446288+0j)
## Error (%) =  0.0136777726317
---
>> From LAL: N210 =  3.08302847349
>> From  PY: N210 =  (3.08263949968+0j)
## Error (%) =  0.0126182061808
---
>> From LAL: N330 =  3.86883702456
>> From  PY: N330 =  (3.86855912206+0j)
## Error (%) =  0.00718361779841
---
>> From LAL: N320 =  0.885187500331
>> From  PY: N320 =  (0.88480304824+0j)
## Error (%) =  0.0434505838557
---
>> From LAL: N440 =  2.58921243577
>> From  PY: N440 =  (2.58831596037+0j)
## Error (%) =  0.0346354701253
---
>> From LAL: N430 =  0.580613263678
>> From  PY: N430 =  (0.580153527625+0j)
## Error (%) =  0.0792438606278
---
>> From LAL: N550 =  1.84464760922
>> From  PY: N550 =  (1.84258780787+0j)
## Error (%) =  0.111788504367


## Compare Spheroidal Harmonic Functions

In [ ]:

#
for l,m,n in ns.lmn:

    #
    slal = lalsim.SpinWeightedSpheroidalHarmonic( xf, l, m, n, inclination, phi_ref )

    #
    spy = slm( xf,l,m,n,inclination,phi_ref )

    #
    print '---'
    print '>> From LAL: S%i%i%i(theta,phi) = '%(l,m,n),slal
    print '>> From  PY: S%i%i%i(theta,phi) = '%(l,m,n),spy
    print '## Error (%) = ',100*abs((slal-spy)/abs(spy))


In [ ]:

#
l,m,n = 3,2,0
jf = 0.8

#
theta = linspace(0,pi,5e2)
phi = 0

#
Spy = slm( jf,l,m,n,theta,phi )
Slal = array( [ lalsim.SpinWeightedSpheroidalHarmonic( jf, l, m, n, th, phi ) for th in theta ] )

#
plot( theta, abs(Spy), label = 'Py' )
plot( theta, abs(Slal), label = 'LAL' )

#
legend( frameon=False )
xlabel(r'$\theta$')
ylabel(r'$|S_{lmn}(\theta,\phi)|$')

#
print 2*pi*trapz(Spy*Spy.conj()*sin(theta),theta )
print 2*pi*trapz(Slal*Slal.conj()*sin(theta),theta )

print Slal

## Compare LAL and Python Models

In [ ]:

#
l,m,n = 4,4,0

#
pyh = physh( ns.meval_mode(l,m,n,eta)(t_code), M_Sol, D_Mpc )

#
lah_raw = lalsim.SimRingdownGenerateSingleBareModeMMRDNS_time( 0.0, deltaT,Mf,xf,eta,l,m,n,D,0,0,len(t_code), physt(0,M_Sol) )
lah = lah_raw.data.data

#
t = physt(t_code,M_Sol)

#
fig = figure( figsize = 6*array([2.1,1]) )

#
def plotc(c,color='k',alpha=1,label=None,linestyle='-'):
    #
    subplot(1,2,1)
    alpha = 0.6
    plot( t, c.real, color = color, alpha=alpha, label=label,linestyle=linestyle )
    plot( t, c.imag, color = color, alpha=alpha,linestyle=linestyle )
    plot( t, abs(c), color = color, alpha=alpha,linestyle=linestyle )
    plot( t,-abs(c), color = color, alpha=alpha,linestyle=linestyle )
    xlim( lim(t) )
    legend( frameon=False )
    #
    subplot(1,2,2)
    alpha = 0.6
    plot( t, unwrap(angle(c)), color = color, alpha=alpha, label=label,linestyle=linestyle )
    xlim( lim(t) )
    legend( frameon=False )

    
plotc( pyh, color='b', label = 'KERR',linestyle='-' )
plotc( lah, color='g', label = 'LAL',linestyle='-' )




## Compare Recomposed Waveforms

In [ ]:

# Format NR Waveform 
hnr = physh( yrd.recompose(inclination,phi_ref,kind='strain',domain='time').wfarr, M_Sol, D_Mpc )

# Eval python model 
hpy = physh( ns.meval( inclination, phi_ref, eta, kind='strain' )(t_code), M_Sol, D_Mpc )

# Evaluate LAL model
hlal_raw = lalsim.SimRingdownMMRDNS_time(0.0,deltaT,M,xf,eta,inclination,phi_ref,D,nonGRparams=None)
chlal = hlal_raw[0].data.data + 1j*hlal_raw[1].data.data

In [ ]:
#
fig = figure( figsize=5*array([3.2,1]) )

#
def plotnr():
    t = hnr[:,0]
    # Plot NR Waveform
    nr_alpha = 0.3
    # Shift time to align with LAL verision
    plot( t-t[0], hnr[:,1], 'k', alpha=nr_alpha )
    plot( t-t[0], hnr[:,2], 'k', alpha=nr_alpha )
    chnr = hnr[:,1]+1j*hnr[:,2]
    plot( t-t[0], abs(chnr), 'k', alpha=nr_alpha, label='NR' )
    plot( t-t[0],-abs(chnr), 'k', alpha=nr_alpha )
    if False:
        ll,mm = 2,2
        hlallmraw = lalsim.SimRingdownGenerateSingleBareModeMMRDNS_time( 0.0, deltaT,M,xf,eta,ll,mm,0,D,0,0,len(t),0.0 )
        hlal22 = sYlm(-2,ll,mm,inclination,phi_ref)*hlallmraw.data.data
        plot( t-t[0],abs(hlal22),'--r',label='LAL (2,2,0) ONLY')
        plot( t-t[0],-abs(hlal22),'--r' )
    xlim( lim(t-t[0]) )
    #
    legend( frameon=False )

#
def plotlal():
    alpha = 0.8
    t = deltaT * arange( len(chlal) )
    # Plot LAL Waveform
    plot( t, abs(chlal), 'b', label='MMRDNS (LAL)',alpha=alpha )
    plot( t,-abs(chlal), 'b',alpha=alpha )
    xlim( lim(t) )
    #
    legend( frameon=False )
    
#
def plotpy():
    alpha=0.90
    color = 'k'
    t = physt(t_code-t_code[0],M_Sol)
    plot( t, hpy.real, '--',alpha=alpha, color=color, label='MMRDNS (PY)'  )
    plot( t, hpy.imag, '--',alpha=alpha, color=color  )
    plot( t, abs(hpy), '--',alpha=alpha, color=color  )
    plot( t,-abs(hpy), '--',alpha=alpha, color=color  )
    #
    legend( frameon=False )

#
subplot(1,3,1)
plotnr()
plotpy()
#
subplot(1,3,2)
plotlal()
#
subplot(1,3,3)
plotnr()
plotlal()
plotpy()


In [ ]:
# ?physh